# Project Euler #31: Coin Sums

## Problem Statement

In England the currency is made up of pound, £, and pence, p, and there are eight coins in general circulation:
1p, 2p, 5p, 10p, 20p, 50p, £1 (100p) and £2 (200p).

How many different ways can £2 be made using any number of coins?

## Journey: From Brute Force to Dynamic Programming

## Initial Attempt (2016)

My first approach attempted to generate all possible combinations through recursion:

In [ ]:
# Original 2019 implementation
import logging
import math

RESULTS = {}
HEY = {}

def _permutations(number, coins_available, paths=None):
    """ Brute-force method """
    if paths is None: paths = [[]]
    
    if number < 0: return False, None
    if number == 0: return True, []
    if number in RESULTS: return True, RESULTS[number]
    
    result = []
    for coin in coins_available:
        valid, path_generated = _permutations(number - coin, coins_available, paths)
        if not valid: break
        result.extend(path + [coin] for path in path_generated)
        if not path_generated: result.append([coin])
    
    logging.debug(f'Finished {number}')
    RESULTS[number] = result
    return True, result

### Critical Evaluation of Initial Approach

**Shortcomings identified:**

1. **Conceptual confusion**: Named the function `_permutations` but was actually trying to solve a combinations problem
2. **Inefficient data structures**: Generating all paths explicitly rather than just counting
3. **Poor naming conventions**: Variables like `HEY` and function names like `better()` lack clarity
4. **Overengineered returns**: Using `(valid, result)` tuples when just the result would suffice
5. **Missing optimization**: Not recognizing this as a classic dynamic programming problem

## The Connection: Climbing Stairs Pattern

After working on similar recursion problems (particularly the climbing stairs problem), I recognized the pattern:

- **Climbing Stairs**: Count ways to reach step n using steps of size {1, 2}
- **Coin Sums**: Count ways to make amount n using coins of denomination {c1, c2, ...}

Both follow the same recursive structure:
- Base case: One way to make 0 (use nothing)
- Recursive case: Sum of ways when including/excluding each option

The key difference:
- **Climbing stairs** counts **permutations** (order matters)
- **Coin sums** counts **combinations** (order doesn't matter)

## First Attempt (2019)

In [ ]:
HEY = {}


def digit_sum(number, coins, max_coin):
	available_coins = sorted(list(filter(lambda coin_: coin_ <= max_coin, coins)))

	result = []
	for coin in available_coins:
		if coin == 1:
			result.append([1] * number)
			continue

		upper_range = math.floor(number / coin) + 1

		for multiplier in range(1, upper_range):
			remainder = number - coin * multiplier
			if remainder < 0: break
			valid, possibles = better(remainder, coins, coins[coins.card_at(coin) - 1])
			if not valid: return result
			result.extend([coin] * multiplier + possible for possible in possibles)
			if not possibles: result.append([coin] * multiplier)

	return result


def better(number, coins, max_coin=None):
	if max_coin is None: max_coin = max(coins)

	available_coins = sorted(list(filter(lambda c: c <= max_coin, coins)))
	if not available_coins: return False, [[]]
	if number == 0: return True, [[]]

	memoization_key = (number, max_coin)
	if memoization_key in HEY: return True, HEY[memoization_key]

	result = digit_sum(number, coins, max_coin)
	# print(f'(number, max_coin)={memoization_key}, result={result}')

	HEY[memoization_key] = result
	return True, result


# noinspection PyDefaultArgument
def coin_sums_legacy(coin_total, coins_available):
	_, paths = better(coin_total, sorted(coins_available))

	unique_combinations = {tuple(sorted(path)) for path in paths}

	return len(unique_combinations)


def combinations_debug(number, coins_available, reverse=True):
	_, paths = better(number, sorted(coins_available))

	unique_combinations = {tuple(sorted(path, reverse=reverse)) for path in paths}
	for path in sorted(unique_combinations, reverse=reverse): print(path)

	return len(unique_combinations), sorted(unique_combinations, reverse=reverse)

## Modern Solution: Dynamic Programming

With better understanding of DP patterns, here's the optimized approach:

In [ ]:
def coin_sums(target: int, coins: list[int]) -> int:
    """
    Count the number of ways to make target amount using given coins.
    
    Uses dynamic programming to build up solutions from smaller amounts.
    Key insight: Process coins systematically to avoid counting duplicates.
    
    Time: O(target * len(coins))
    Space: O(target)
    """
    # dp[i] represents number of ways to make amount i
    dp = [0] * (target + 1)
    dp[0] = 1  # Base case: one way to make 0
    
    # Process each coin denomination
    for coin in coins:
        # Update all amounts this coin can contribute to
        for amount in range(coin, target + 1):
            dp[amount] += dp[amount - coin]
    
    return dp[target]

# Solve Project Euler #31
def solve():
    target = 200  # £2 in pence
    coins = [1, 2, 5, 10, 20, 50, 100, 200]
    return coin_sums(target, coins)

answer = solve()
print(f"Number of ways to make £2: {answer}")

## Alternative Approach: Top-Down with Memoization

For completeness, here's the recursive approach with proper memoization:

In [ ]:
from functools import lru_cache

def coin_sums_recursive(target: int, coins: list[int]) -> int:
    """
    Recursive solution with memoization.
    Process coins in order to ensure we count combinations, not permutations.
    """
    coins_tuple = tuple(coins)  # For hashability in cache
    
    @lru_cache(maxsize=None)
    def count_ways(amount: int, coin_index: int) -> int:
        # Base cases
        if amount == 0:
            return 1
        if amount < 0 or coin_index >= len(coins_tuple):
            return 0
        
        # Include current coin or skip to next
        include = count_ways(amount - coins_tuple[coin_index], coin_index)
        exclude = count_ways(amount, coin_index + 1)
        
        return include + exclude
    
    return count_ways(target, 0)

# Verify with alternative approach
answer_recursive = coin_sums_recursive(200, [1, 2, 5, 10, 20, 50, 100, 200])
print(f"Recursive solution: {answer_recursive}")

## Complexity Analysis

### Bottom-Up DP
- **Time**: O(n × m) where n = target amount, m = number of coins
- **Space**: O(n) for the DP array

### Top-Down Recursive
- **Time**: O(n × m) with memoization
- **Space**: O(n × m) for the cache + O(m) for recursion stack

The bottom-up approach is preferred for better space efficiency.

## Testing with Smaller Examples

Let's verify our solution with smaller, traceable examples:

In [ ]:
def test_coin_sums():
    test_cases = [
        # (target, coins, expected)
        (4, [1, 2], 3),  # {1,1,1,1}, {1,1,2}, {2,2}
        (5, [1, 2, 5], 4),  # {5}, {2,2,1}, {2,1,1,1}, {1,1,1,1,1}
        (10, [2, 5, 10], 2),  # {10}, {5,5}
    ]
    
    for target, coins, expected in test_cases:
        result = coin_sums(target, coins)
        status = "✓" if result == expected else "✗"
        print(f"{status} Target={target}, Coins={coins}: Got {result}, Expected {expected}")

test_coin_sums()

## Evolution Summary

The journey from the 2019 implementation to the current solution demonstrates key improvements:

1. **Pattern Recognition**: Identifying this as a classic DP problem similar to climbing stairs
2. **Clarity**: Clean, self-documenting code with proper naming
3. **Efficiency**: From generating all paths to just counting (exponential space → linear space)
4. **Understanding**: Clear distinction between combinations and permutations
5. **Simplicity**: Removed unnecessary complexity while maintaining correctness

The modern solution is not just more efficient—it's more maintainable and demonstrates a deeper understanding of the underlying algorithmic patterns.